# Lab_07: Classification


In [1]:
# !wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip -O free_spoken_digit.zip

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import os
root = './free_spoken_digit/'
dev = root + 'dev/'
dev_samples = os.listdir(dev)


In [4]:
import scipy.io.wavfile as spwav

# Return the sample rate (in samples/sec) and data from an LPCM WAV file.
spwav.read(dev + dev_samples[0])

(8000, array([ 362,  360,  356, ..., -168,  -56,  318], dtype=int16))

In [5]:
def min_max_avg(ary: np.array) -> tuple:
    return (np.min(ary), np.max(ary), np.mean(ary))

In [6]:
d = dict()
frequency = spwav.read(dev + dev_samples[0])[0]
lengths = []
mma_list = [] # min_max_avg list
arys = []
for sample in dev_samples:
    id = int(sample.split('_')[0])
    ary = spwav.read(dev + sample)[1]
    time_length = ary.shape
    arys.append(ary)
    lengths.append(time_length)
    mma_list.append(min_max_avg(ary))
    d_temp = {'label': int(sample.split('_')[1][:-4]),
              'ary': ary,
              'min_max_avg':min_max_avg(ary),
              'time': round(time_length[0]/frequency, 2)} # till -4 since it has the .wav extension
    d[id] = d_temp 
d

{481: {'label': 8,
  'ary': array([ 362,  360,  356, ..., -168,  -56,  318], dtype=int16),
  'min_max_avg': (-11989, 7846, 0.09351203167834297),
  'time': 0.41},
 863: {'label': 1,
  'ary': array([ 3,  6, -2, ..., 13,  6,  6], dtype=int16),
  'min_max_avg': (-2957, 1874, -1.7005242463958061),
  'time': 0.38},
 845: {'label': 2,
  'ary': array([-21,  31, -34, ...,  -4,   9,  34], dtype=int16),
  'min_max_avg': (-1227, 1604, 0.10849233071455294),
  'time': 0.33},
 1146: {'label': 3,
  'ary': array([  0,   8, -17, ...,  19,   9,   2], dtype=int16),
  'min_max_avg': (-490, 592, -0.06704980842911877),
  'time': 0.33},
 307: {'label': 2,
  'ary': array([   0,    0, -256, ..., -512, -512, -768], dtype=int16),
  'min_max_avg': (-4608, 4608, -230.59830320914793),
  'time': 0.34},
 1281: {'label': 5,
  'ary': array([  0,   0, -14, ..., -12,  -8, -13], dtype=int16),
  'min_max_avg': (-4278, 2311, 0.471718835804503),
  'time': 0.46},
 223: {'label': 6,
  'ary': array([-2,  0, -2, ..., 16, 17, -6],

The method `read()` of `scipy.io.wavfile` returns:
- the sample rate in sample/sec (in our case 8000, hence the frequency is 8kHz)
- `numpy array of dtype=int16`. According to documentation, it is a format of 16-bit integer PCM with min = -32767 and max = 32767

In [7]:
for length in lengths:
    print(round(length[0]/8000, 2), '\tsec')

0.41 	sec
0.38 	sec
0.33 	sec
0.33 	sec
0.34 	sec
0.46 	sec
0.27 	sec
0.34 	sec
0.32 	sec
0.22 	sec
0.48 	sec
0.25 	sec
0.47 	sec
0.32 	sec
0.34 	sec
0.39 	sec
0.37 	sec
0.37 	sec
0.69 	sec
0.4 	sec
0.41 	sec
0.4 	sec
0.28 	sec
0.54 	sec
0.31 	sec
0.41 	sec
0.33 	sec
0.23 	sec
0.6 	sec
0.51 	sec
0.33 	sec
0.42 	sec
0.36 	sec
0.25 	sec
0.42 	sec
0.61 	sec
0.5 	sec
0.37 	sec
0.45 	sec
0.6 	sec
0.66 	sec
0.43 	sec
0.46 	sec
0.25 	sec
0.49 	sec
0.52 	sec
0.63 	sec
0.5 	sec
0.26 	sec
0.41 	sec
0.28 	sec
0.39 	sec
0.47 	sec
0.33 	sec
0.48 	sec
0.38 	sec
0.37 	sec
0.3 	sec
0.45 	sec
0.51 	sec
0.18 	sec
0.58 	sec
0.44 	sec
0.72 	sec
0.37 	sec
0.43 	sec
0.21 	sec
0.41 	sec
0.36 	sec
0.47 	sec
0.36 	sec
0.27 	sec
0.39 	sec
0.35 	sec
0.58 	sec
0.27 	sec
0.37 	sec
0.48 	sec
0.53 	sec
0.3 	sec
0.49 	sec
0.17 	sec
0.39 	sec
0.41 	sec
0.16 	sec
0.34 	sec
0.29 	sec
0.44 	sec
0.35 	sec
0.44 	sec
0.21 	sec
0.49 	sec
0.26 	sec
0.54 	sec
0.2 	sec
0.28 	sec
0.51 	sec
0.33 	sec
0.28 	sec
0.51 	sec
0.49 	sec

As we can see, all the samples are below 1 sec length but they are not aligned (they don't have the same length). If we want to align them, we can paddle each array with a neutral value.
I now want to check whether if conditions where the same. The idea that I've got is to check min, max and avg of each sample.

In [8]:
print('min\tmax\tavg')
for mma in mma_list:
    print(f'{mma[0]}\t{mma[1]}\t{round(mma[2],2)}')

min	max	avg
-11989	7846	0.09
-2957	1874	-1.7
-1227	1604	0.11
-490	592	-0.07
-4608	4608	-230.6
-4278	2311	0.47
-2603	1449	-0.83
-12032	8960	-229.93
-1188	906	-0.19
-519	833	-0.4
-13391	9945	-0.34
-7680	4864	-229.34
-2877	1884	0.06
-5120	3072	-230.7
-1885	2069	0.08
-3059	1272	-0.85
-7936	6656	-231.37
-1782	1141	-0.58
-25254	23473	-0.16
-10183	10571	0.19
-556	925	-0.15
-14520	9727	-0.46
-8192	5376	-231.85
-10516	8765	0.6
-5888	3328	-229.09
-14776	13650	0.29
-3832	1704	0.35
-6656	4608	-230.37
-15747	17403	0.3
-12407	10085	-0.33
-668	835	0.16
-13824	7424	-251.46
-8192	6400	-231.17
-4352	4608	-230.31
-783	834	-0.1
-805	994	0.11
-9216	6144	-253.75
-7936	7168	-229.54
-1808	1157	-0.55
-11205	7531	-1.13
-10605	8792	0.52
-15871	16045	0.36
-874	1086	-0.12
-730	868	-0.34
-14182	12363	-0.74
-25136	31504	0.08
-24071	24044	0.36
-845	664	-0.08
-2304	2048	-229.45
-566	803	0.13
-1117	973	-1.08
-1836	1130	-0.25
-13891	12911	0.66
-8960	5376	-228.02
-11530	9447	-0.67
-8960	5632	-227.91
-8448	5376	-248.4
-28

There are some values where the average is strongly negative, if we try to hear them we can see that they are noisier than the others.

_We can try to remove them and then classify_

For what concern the alignment, since I don't want to change the average, I shall add 0 as padding element.

In [9]:
# I will align them to such value
max_length = max(lengths)[0]
max_length

17567

In [10]:
# Check if all samples are 1D
for l in lengths:
    if len(l) != 1:
        print(l)
# They all have the same shape

In [11]:
def padding(ary: np.array, target_len:int, padding_element:int=0) -> np.array:
    return np.pad(ary, (0, target_len), 'constant',  constant_values=padding_element)

In [85]:
new_arys = []
for a in arys:
    width_to_pad = max_length-len(a)
    new_ar = padding(a, width_to_pad, 0)
    # new_ar = padding(a, width_to_pad, np.mean(a)) try with this one
    new_arys.append(new_ar)

what if we pad with mean?


In [86]:
# Check if they are all padded with the same length
for array in new_arys:
    if len(array) != max_length:
        print('Found one')

In [87]:
new_mma_list = []
for a in new_arys:
    new_mma_list.append(min_max_avg(a))

print('min\tmax\tavg')
for mma in new_mma_list:
    print(f'{mma[0]}\t{mma[1]}\t{round(mma[2],2)}')

min	max	avg
-11989	7846	0.02
-2957	1874	-0.3
-1227	1604	0.02
-490	592	-0.01
-4608	4608	-35.59
-4278	2311	0.1
-2603	1449	-0.1
-12032	8960	-35.09
-1188	906	-0.03
-519	833	-0.04
-13391	9945	-0.07
-7680	4864	-26.58
-2877	1884	0.01
-5120	3072	-33.49
-1885	2069	0.01
-3059	1272	-0.15
-7936	6656	-38.6
-1782	1141	-0.1
-25254	23473	-0.05
-10183	10571	0.03
-556	925	-0.03
-14520	9727	-0.08
-8192	5376	-29.52
-10516	8765	0.15
-5888	3328	-32.38
-14776	13650	0.05
-3832	1704	0.05
-6656	4608	-24.37
-15747	17403	0.08
-12407	10085	-0.08
-668	835	0.02
-13824	7424	-48.38
-8192	6400	-37.86
-4352	4608	-26.52
-783	834	-0.02
-805	994	0.03
-9216	6144	-57.4
-7936	7168	-38.56
-1808	1157	-0.11
-11205	7531	-0.31
-10605	8792	0.16
-15871	16045	0.07
-874	1086	-0.03
-730	868	-0.04
-14182	12363	-0.17
-25136	31504	0.02
-24071	24044	0.1
-845	664	-0.02
-2304	2048	-26.83
-566	803	0.02
-1117	973	-0.14
-1836	1130	-0.04
-13891	12911	0.14
-8960	5376	-34.44
-11530	9447	-0.15
-8960	5632	-39.73
-8448	5376	-41.42
-2816	2304	-31.56
-

**NOTE**: If we pad with 0, then if we didn't remove the strongly negative values their value will go towards 0 mean. Otherwise if we pad using mean value, the mean value remains the same. What can we do? We can remove strongly negative mean values before and then apply the same preprocessing or we can add the mean value. By now I don't know what is right.

NOTA: provare bucketing in cui inserisco un parametro per vedere in quanti bucket dividere. Dopodiché usare una moving average come valore di ogni bucket. Provare con moving average in cui al bucket 1 media di bucket 1, bucket 2 media dei valori fino al termine di bucket 2, e così via.
Un'alternativa è usare la exponential moving average (guarda wikipedia)

In [88]:
df = pd.DataFrame.from_dict(d).T
df.ary = new_arys
df.min_max_avg = new_mma_list
df.sort_index(axis=0, inplace=True)
df.head()

,label,ary,min_max_avg,time
0,4,"[14, 21, 0, 18, -11, 22, -15, -53, 9, -14, 11,...","(-8198, 10350, -0.009221836397791313)",0.39
1,7,"[-2, 12, 1, -18, 7, -7, 1, -1, 0, 0, -6, -9, 1...","(-1885, 1110, -0.02134684351340582)",0.41
2,5,"[-9, -7, -5, -8, -13, -1, 20, -12, 6, 15, 3, 2...","(-3789, 1686, -0.0987078044059885)",0.42
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34
4,3,"[-20, 10, 26, -13, 22, -16, 11, 7, -21, 1, 63,...","(-566, 835, 0.0005692491603574884)",0.24


In [89]:
avg = []
for tup in df.min_max_avg:
    avg.append(tup[-1])
df['avg'] = avg
df.head()

,label,ary,min_max_avg,time,avg
0,4,"[14, 21, 0, 18, -11, 22, -15, -53, 9, -14, 11,...","(-8198, 10350, -0.009221836397791313)",0.39,-0.009222
1,7,"[-2, 12, 1, -18, 7, -7, 1, -1, 0, 0, -6, -9, 1...","(-1885, 1110, -0.02134684351340582)",0.41,-0.021347
2,5,"[-9, -7, -5, -8, -13, -1, 20, -12, 6, 15, 3, 2...","(-3789, 1686, -0.0987078044059885)",0.42,-0.098708
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34,-35.615871
4,3,"[-20, 10, 26, -13, 22, -16, 11, 7, -21, 1, 63,...","(-566, 835, 0.0005692491603574884)",0.24,0.000569


In [90]:
grpby_label = df.groupby(['label'])
grpby_label.describe()

avg                                                                  \
       count       mean        std        min        25%       50%       75%   
label                                                                          
0      150.0 -11.860604  21.813232 -64.601127  -0.297547 -0.045511  0.033856   
1      150.0  -8.784752  14.921597 -42.042466 -26.099846 -0.095235 -0.007144   
2      150.0  -8.259869  14.316664 -44.403256 -19.727899 -0.059885  0.002448   
3      150.0  -7.431707  13.142491 -45.146468 -14.561251 -0.067285 -0.003017   
4      150.0  -8.556077  15.216685 -48.439916  -0.248420 -0.015569  0.046251   
5      150.0  -9.397362  17.186430 -54.604201  -0.380372 -0.041356  0.013591   
6      150.0  -7.647978  13.276041 -48.687653 -16.795127 -0.064923 -0.007030   
7      150.0  -9.727759  16.604224 -47.172084 -26.544316 -0.057750  0.006134   
8      150.0  -9.643413  18.249297 -53.831844  -0.153484 -0.047134  0.001665   
9      150.0 -12.146398  21.754845 -61.584562  -0.219901 -0.056754  0.018614   

                 
            max  
label            
0      0.430979  
1      0.291854  
2      0.258553  
3      0.195366  
4      0.502249  
5      0.545568  
6      0.183640  
7      0.174532  
8      0.205100  
9      0.312745

In [91]:
mask = df.label == 2
df[mask]

,label,ary,min_max_avg,time,avg
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34,-35.615871
17,2,"[-256, -256, -256, -256, -256, -256, -256, -25...","(-6144, 4864, -37.13143963112655)",0.36,-37.131440
39,2,"[-942, 867, -154, 123, 280, -402, 991, -654, 2...","(-8931, 7994, -0.01616667615415267)",0.51,-0.016167
60,2,"[-59, -16, -32, 8, 3, 18, 15, 16, 6, -1, 10, 5...","(-2035, 1308, -0.15130642682302042)",0.31,-0.151306
67,2,"[0, 0, 2, -2, 2, -2, 2, -3, 4, -4, 0, 0, -3, 0...","(-2665, 1363, -0.1060511185746001)",0.33,-0.106051
...,...,...,...,...,...
1460,2,"[-5, -6, -8, -18, 4, -7, 2, 0, -5, -2, -2, -14...","(-642, 702, 0.003017020549894689)",0.27,0.003017
1467,2,"[3, -3, 7, 0, 6, -9, 6, 2, 12, 4, 7, 8, -9, 3,...","(-917, 1330, -0.025388512551943986)",0.24,-0.025389
1474,2,"[-6, 8, -4, 9, 3, 12, 6, 14, -15, 6, -1, 15, 9...","(-530, 744, 0.02117606876529857)",0.27,0.021176
1477,2,"[-12, 6, 11, 3, -1, 4, -2, -1, -2, 2, -5, -1, ...","(-602, 729, -0.06728525075425514)",0.53,-0.067285


In [93]:
# mask1 = df.avg < -10 before zero padding. Ok with mean padding
mask1 = df.avg < - 5 # this is good with 0 padding. Still 372 lines
df[mask1]

,label,ary,min_max_avg,time,avg
3,2,"[-256, -512, 0, -256, 256, -256, 0, -512, 0, -...","(-4864, 4096, -35.615870666590766)",0.34,-35.615871
5,4,"[-256, -512, -256, -512, -256, -256, 0, -256, ...","(-14848, 9472, -32.278704388911024)",0.31,-32.278704
14,3,"[0, 0, -256, 0, -256, 0, -256, 0, -256, -256, ...","(-5376, 5376, -35.89275345818865)",0.34,-35.892753
17,2,"[-256, -256, -256, -256, -256, -256, -256, -25...","(-6144, 4864, -37.13143963112655)",0.36,-37.131440
18,7,"[-256, -512, 0, -512, 0, -256, -512, 0, -512, ...","(-4864, 3584, -37.728923549837766)",0.36,-37.728924
...,...,...,...,...,...
1472,8,"[0, -256, 0, -256, -256, 0, -256, 0, -256, 0, ...","(-17664, 11008, -47.070074571640006)",0.4,-47.070075
1473,4,"[-256, -256, 0, 0, 0, -256, 0, -256, 0, -256, ...","(-16640, 12544, -48.43991575112427)",0.46,-48.439916
1480,6,"[-256, -512, -256, -512, 0, -256, -256, 0, -51...","(-6656, 5120, -25.808390732623668)",0.25,-25.808391
1483,6,"[-768, -1280, -1536, -1536, -1792, -1792, -153...","(-5632, 4608, -26.65361188592247)",0.24,-26.653612


there are around 370 samples with pretty low avg. 372/1500 are very noisy.

# Bucketing

In [97]:
# portion = .10
# num_buckets = round(portion * max_length)
num_buckets = 10 #divided into 10 buckets

In [101]:
prova = np.array_split(df.ary[0], num_buckets)
cnt = 0
for a in prova:
    print(np.mean(a))
    cnt += 1
# print(cnt)

-0.5167899829254411
0.42458736482640863
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [110]:
# We want to divide each array in a set of features.
# To do this, we create a new dictionary in which we will have
# the id and the set of features.
# By changing the parameter num_buckets we can increase or decrease
# the number of features that we will use

# portion = .10
# num_buckets = round(portion * max_length)
num_buckets = 10 #divided into 10 buckets

another_dict = dict()

for id, array in zip(df.ary.index, df.ary):
    split_ary = np.array_split(array, num_buckets)
    cnt_feature = 0
    features = dict()
    for subary in split_ary:
        features[f'feature_{cnt_feature}'] = subary
        cnt_feature += 1
    another_dict[id] = features

another_dict[0]

{'feature_0': array([  14,   21,    0, ..., -257, -153,  -79], dtype=int16),
 'feature_1': array([ -75, -184, -458, ...,    0,    0,    0], dtype=int16),
 'feature_2': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_3': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_4': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_5': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_6': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_7': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_8': array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 'feature_9': array([0, 0, 0, ..., 0, 0, 0], dtype=int16)}

In [187]:
def simple_moving_average(list_of_arys:list) -> np.array:
    list_of_arys = list(list_of_arys)
    ary = np.zeros(shape=(len(list_of_arys)))
    for id in range(len(list_of_arys)):
        num_values = len(list_of_arys[id])
        tot = np.sum(list_of_arys[id])
        ary[id] = tot/num_values
    return ary

def cumulative_moving_average(list_of_arys:list) -> np.array:
    temp = [(np.sum(i), len(i)) for i in list_of_arys]
    mean_variable = 0
    values_cnt = 0
    out_list = []
    
    for i in range(len(temp)):
        sum, l = temp[i]
        if i == 0:
            mean_variable = sum/l
            values_cnt += l
            out_list.append(mean_variable)
        else:
            mean_variable = (sum + values_cnt*mean_variable)/(l + values_cnt)
            values_cnt += l
            out_list.append(mean_variable)


    return np.array(out_list)

### Simple moving average (SMA)

In [196]:
# SMA
another_dict_avg = {}
for key, item in another_dict.items():
    simple_mean = simple_moving_average(item.values())
    another_dict_avg[key] = {f'feature_{i}':m for i,m in enumerate(simple_mean)}
another_dict_avg[10]

{'feature_0': -1.4314171883892999,
 'feature_1': 0.055776892430278883,
 'feature_2': 0.0,
 'feature_3': 0.0,
 'feature_4': 0.0,
 'feature_5': 0.0,
 'feature_6': 0.0,
 'feature_7': 0.0,
 'feature_8': 0.0,
 'feature_9': 0.0}

### Cumulative moving average
In a cumulative average (CA), the data arrive in an ordered datum stream, and the user would like to get the average of all of the data up until the current datum. For example, an investor may want the average price of all of the stock transactions for a particular stock up until the current time. As each new transaction occurs, the average price at the time of the transaction can be calculated for all of the transactions up to that point using the cumulative average, typically an equally weighted average of the sequence of _n_ values $x_{1}, \ldots ,x_{n}$ up to the current time:

$$
CA_n = \frac{x_1+\dots+x_n}{n}
$$

it is possible to simply update cumulative average as a new value, 
$x_{n+1}$ becomes available, using the formula:

$$
CA_n = \frac{x_{n+1}+n\cdot CA_n}{n+1} \to CA_n = \frac{x_{n+1} + \dots + x_{n+n_1}+n\cdot CA_n}{n+n_1}
$$

In [192]:
# Cumulative average
cum_avg_dict = dict()
for key, item in another_dict.items():
    mean = cumulative_moving_average(item.values())        
    cum_avg_dict[key] = {f'feature_{i}': m for i,m in enumerate(mean)}
cum_avg_dict[10]

{'feature_0': -1.4314171883892999,
 'feature_1': -0.6878201479795105,
 'feature_2': -0.4585467653196737,
 'feature_3': -0.34391007398975526,
 'feature_4': -0.2751280591918042,
 'feature_5': -0.22927338265983685,
 'feature_6': -0.19652004227986014,
 'feature_7': -0.1719672714336535,
 'feature_8': -0.15286825627727532,
 'feature_9': -0.13758752205840497}

### Exponential weighted average
